# Lesson 4:  Grouping & Joining Data;  Visualizing Distributions

Adapted from material by Ani Adhikari, Suraj Rampure, and Fernando Pérez and Josh Hug and Narges Norouzi


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Dataset: Election data


In [ ]:
elections = pd.read_csv('data/elections.csv')

## Dataset: Colorado baby names

In today's lesson, we'll work with data from US Social Security applications (https://www.ssa.gov/oact/babynames/background.html https://www.ssa.gov/oact/babynames/) to download 
the `babynames` dataset, which contains information about the names of infants born in Colorado.

The cell below pulls census data from a government website and then loads it into a usable form. The code shown here is outside of the scope of CSCI 3022, but you're encouraged to dig into it if you are interested!

In [ ]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "data/babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

state_name = 'CO.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(state_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames

In [ ]:
babynames.describe()

## Grouping


To illustrate Panda's grouping function, we're going to look at a smaller subset of names for demonstration purposes only:

In [ ]:

babynames.head()

#Select a random subset of names from this dataset
babynames_temp=babynames.iloc[[5,15,49100, 49000, 55502, 55503]].sort_values(by="Name").reset_index().drop(columns="index")

In [ ]:
babynames_temp

### GroupBy

We can group rows that share a common feature, then aggregate data across the group.

Syntax  `dataframe.groupby(column_names).agg(aggregation_function)`


In this example, we count the total number of babies born in each year 

Calling `.groupby()` generates DataFrameGroupBy objects → "mini" sub-DataFrames


Each subframe contains all rows that correspond to the same group 


We cannot work directly with DataFrameGroupBy objects! The code below is to help understand what goes on conceptually – in reality, we can't "see" the result of calling .groupby.

It turns out we can print out the subframes that result. This isn't something you'll do for any practical purpose. However, it may help you get an understanding of what groupby is actually doing.


### `DataFrameGroupBy` Objects

The result of `groupby` is not a `DataFrame` or a list of `DataFrame`s. It is instead a special type called a `DataFrameGroupBy`.

In [ ]:
grouped_by_year = babynames_temp.groupby("Year")
type(grouped_by_year)

`GroupBy` objects are structured like dictionaries. In fact, we can actually see the dictionaries with the following code:

In [ ]:
grouped_by_year.groups

The `key`s of the dictionary are the groups (in this case, `Year`), and the `value`s are the **indices** of rows belonging to that group. We can access a particular sub-`DataFrame` using `get_group`:

In [ ]:
grouped_by_year.get_group(2008)

In [ ]:
# This is just to demonstrate under the hood what the groupby call does - this
# is not something you would usually run
for n, g in grouped_by_year:
    display(g)

Once we've formed groups, we can aggregate each sub-dataframe (a.k.a. group) into a single row using an aggregation function. 

There are many different aggregation functions we can use, all of which are useful in different applications.


For example, if we use `.agg(min)` on the groups above, we get back a single DataFrame where each group has been replaced by a single row. In the each new column for the aggregate row, the values that appear are the minimum column values for all values in that group.


In [ ]:
babynames_temp.groupby("Year").agg({"Count":"sum"})

In [ ]:
babynames_temp.groupby("Year")[["Count"]].agg("sum")

In [ ]:
babynames_temp.groupby("Year").first()

In [ ]:
babynames_temp.sort_values(by="Count").groupby("Year").first()

In [ ]:
# What is the earliest year in which each name appeared?

babynames_temp.groupby("Name")[["Year"]].agg(min)

In [ ]:
# What is the largest single-year count of each name?
babynames_temp.groupby("Name")[["Count"]].agg(max)

In [ ]:
# What is the earliest the name appeared AND the largest single-year count of each name?

babynames_temp.groupby("Year").agg({"Count":max, "Name":min}).rename(columns={"Count":"MaxCount", "Name":"MinName"})

In [ ]:
babynames_temp.groupby("Year").agg({"Count":[max, min, sum]})




### `groupby.size` and `groupby.count()`

In [ ]:
df = pd.DataFrame({'letter':['A', 'A', 'B', 'C', 'C', 'C'], 
                   'num':[1, 2, 3, 4, np.NaN, 4], 
                   'state':[np.NaN, 'tx', 'fl', 'hi', np.NaN, 'ak']})
df

`groupby.size()` returns a `Series`, indexed by the `letter`s that we grouped by, with values denoting the number of rows in each group/sub-DataFrame. It does not care about missing (`NaN`) values.

In [ ]:
df.groupby("letter").size()

`groupby.count()` returns a `DataFrame`, indexed by the `letter`s that we grouped by. Each column represents the number of non-missing values for that `letter`.

In [ ]:
df.groupby("letter").count()

You might recall `value_counts()` function we talked about last week. What's the difference?

In [ ]:
df["letter"].value_counts()

Turns out `value_counts()` does something similar to `groupby.size()`, except that it also sorts the resulting `Series` in descending order.

---

### Practice Exercises


**Practice:  Write code to calculate the total number of babies in Colorado with each name.**

In [ ]:
...


**Practice:  How many babies in the Colorado dataset have your name?**

In [ ]:
...

### `Groupby` with multiple columns

**Practice:  Suppose we want to build a table showing the total number of babies born of each sex in each year. One way is to groupby using both columns of interest.  Try this in the cell below:**


In [ ]:
...

## Joining Tables

Suppose we want to know the popularity of presidential candidates' first names out of babies born in Colorado in 2023?

First, we'll create a new dataframe with data just from 2023

In [ ]:
babynames_2023 = babynames[babynames["Year"]==2023]
babynames_2023.head(10)

### `.str` Functions for Defining a Condition

We're interested in joining with the elections dataset on the First Name of the Candidate.   To grab the first name of each Candidate we can use a `.str` function (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html#pandas.Series.str.split)

In [ ]:
elections["Candidate"].str.split().str[0]

**Let's add a column to the elections dataframe with just the first name of the candidate**

In [ ]:
elections=elections.assign(First_Name =  elections["Candidate"].str.split().str[0])

elections


**Practice:  Create a new DataFrame merged, which merges the election dataset and the babynames_2023 dataset.**

In [ ]:
merged = ...
merged

**Practice:  What were the most popular presidential names of babies born in Colorado in 2023?**

In [ ]:
...

### Joining Tables on Indices:

What if you want to join your tables on index values?  

For example: suppose we set `First Name` as the index in our `elections` DataFrame
and we set `Name` as the index in our `babynames` DataFrame:

In [ ]:
elections_new = elections.set_index("First Name")
babynames_2023new = babynames_2023.set_index("Name")

Since "First Name" and "Name" are no longer columns, we can't put them as input to the `left_on` or `right_on` argument.  Instead we will set the `left_index` and `right_index` to True:

In [ ]:
merged2 = pd.merge(left = elections_new, right = babynames_2023new, 
                  left_index = True, right_index=True)
merged2

## New Dataset:  Newborns 
Next we'll look at a sample of data of newborns born in the 1960s in Oakland California. [Stat Labs](https://www.stat.berkeley.edu/~statlabs/) by Deborah Nolan and Terry Speed has details about a larger dataset from which this set is drawn.

The data contains the following variables for 1,174 mother-baby pairs: the baby’s birth weight in ounces, the number of gestational days, the mother’s age in completed years, the mother’s height in inches, pregnancy weight in pounds, and whether or not the mother smoked during pregnancy.

In [ ]:
births = pd.read_csv('data/baby.csv')
births

# Visualizing Distributions: Qualitative Variables
## Bar Plots

We often use bar plots to display distributions of a categorical variable:

In [ ]:
babies = births['Maternal Smoker'].value_counts()
babies

In [ ]:
import matplotlib.pyplot as plt

plt.bar(babies.index,babies.values);


In [ ]:
import seaborn as sns

sns.countplot(data = births, x = 'Maternal Smoker');

In [ ]:
import plotly.express as px
px.histogram(births, x = 'Maternal Smoker', color = 'Maternal Smoker')

In [ ]:
births['Maternal Smoker'].value_counts().plot(kind='bar')

# Visualizing Distributions:  Quantitative Variables

In [ ]:
sns.countplot(data = births, x = 'Maternal Pregnancy Weight');

In [ ]:
sns.histplot(data = births, x = 'Maternal Pregnancy Weight', bins = 20);

In [ ]:
px.histogram(births, x = 'Maternal Pregnancy Weight')

In [ ]:
sns.histplot(data = births, x = 'Maternal Pregnancy Weight');

In [ ]:
sns.histplot(data = births, x = 'Maternal Pregnancy Weight', stat= 'density')

In [ ]:
sns.histplot(data = births, x = 'Maternal Pregnancy Weight', stat='density', kde = True);


In [ ]:
p10= np.percentile(births['Maternal Pregnancy Weight'], 10)


births['category'] = None
births.loc[(births['Birth Weight'] <= p10), 'category'] = 'Less than the 10th percentile'
births.loc[(births['Birth Weight'] > p10) , 'category'] = 'Greater than the 10th percentile'

sns.histplot(births, x = 'Birth Weight', hue = 'category', bins = 36, stat='density');

births.drop(columns = ['category'], inplace = True)

#Mark the 10th percentile on the graph
plt.scatter(p10, -.001, marker='^', color='red', s=400)

print(p10)

In [ ]:
q1, median, q3 = np.percentile(births['Birth Weight'], [25, 50, 75])
iqr = q3 - q1

births['category'] = None
births.loc[(births['Birth Weight'] < q1) | (births['Birth Weight'] > q3), 'category'] = 'Outside of the middle 50%'
births.loc[(births['Birth Weight'] > q1) & (births['Birth Weight'] < q3), 'category'] = 'In the middle 50%'

sns.histplot(births, x = 'Birth Weight', hue = 'category', bins = 30, stat = "density");

births.drop(columns = ['category'], inplace = True)


plt.scatter(q1, -.001, marker='^', color='orange', s=400)

plt.scatter(median, -.001, marker='^', color='red', s=400)

plt.scatter(q3, -.001, marker='^', color='green', s=400)

display([q1, median, q3])

## Box Plots

In [ ]:
plt.figure(figsize = (3, 6))
sns.boxplot(data = births, y = 'Birth Weight');

In [ ]:
bweights = births['Birth Weight']
q1 = np.percentile(bweights, 25)
q2 = np.percentile(bweights, 50)
q3 = np.percentile(bweights, 75)
iqr = q3 - q1


q1, q2, q3

## Violin Plots

In [ ]:
plt.figure(figsize = (3, 6))
sns.violinplot(data = births, y = 'Birth Weight');


In [ ]:
#You can put a boxplot inside a violin plot...

px.violin(births, y = "Birth Weight", box=True, width=400)

## Describing Distributions

In [ ]:
median = births['Maternal Pregnancy Weight'].median()
mean = births['Maternal Pregnancy Weight'].mean()

print("Median", median)
print("Mean", mean)